In [234]:
import pandas as pd
import scipy.stats as st
import scipy.linalg as la
import numpy as np
import math
maf = 0.005

In [235]:
european_reference_panel_ld = pd.read_pickle('./reference_panels/european_reference_panel.pickle')
european_reference_panel_map = pd.read_pickle('./reference_panels/european_reference_panel_map.pickle')
european_reference_panel_map.rename(columns={'POS18':'SNP_pos'}, inplace=True)

In [236]:
sample_summary = pd.read_pickle("./EUR_sample.pickle")

In [237]:
sample_summary['T_CDF'] = st.t.logcdf(-abs(sample_summary['T']), 2)
sample_summary['T-P'] = st.t.cdf(-abs(sample_summary['T']), 2)
sample_summary['T_2CDF'] = 2*sample_summary['T_CDF']
sample_summary['T_2CDF2'] = sample_summary['T_2CDF']-math.log(2)
sample_summary['T_PPF'] = sample_summary.apply(lambda x: st.norm.ppf(math.e**x['T_2CDF2']), axis=1)
sample_summary['T-Z'] = sample_summary.apply(lambda x: abs(x['T_PPF']) if x['T'] >= 0 else -abs(x['T_PPF']) , axis=1)
sample_summary

,SNP_id,SNP_pos,Effect_allele,Non_effect_allele,T,T_CDF,T-P,T_2CDF,T_2CDF2,T_PPF,T-Z
0,rs2894066,29001906,A,T,-4.8490,-3.912127,0.019998,-7.824254,-8.517401,-3.540139,-3.540139
1,rs3763338,29002290,A,T,0.8687,-1.434234,0.238298,-2.868469,-3.561616,-1.904956,1.904956
2,rs2071790,29019781,A,T,-4.4720,-3.760589,0.023270,-7.521179,-8.214326,-3.459343,-3.459343
3,rs2071789,29020055,T,A,3.2710,-3.192781,0.041058,-6.385562,-7.078709,-3.140652,3.140652
4,rs2071788,29020286,T,A,1.4930,-1.987786,0.136998,-3.975571,-4.668718,-2.350096,2.350096
...,...,...,...,...,...,...,...,...,...,...,...
1537,rs12191571,33988458,C,G,0.3078,-0.932253,0.393666,-1.864507,-2.557654,-1.422185,1.422185
1538,rs2477223,33993780,A,T,-0.5957,-1.184482,0.305905,-2.368964,-3.062111,-1.676820,-1.676820
1539,rs9368783,33995346,C,G,-1.0610,-1.609745,0.199938,-3.219491,-3.912638,-2.054003,-2.054003
1540,rs12216193,33995958,T,A,1.0650,-1.613365,0.199216,-3.226730,-3.919877,-2.056991,2.056991


In [238]:
european_reference_panel_map_maf_filtered = european_reference_panel_map.query('@maf <= maf <= 1-@maf')
european_reference_panel_ld_maf_filtered = european_reference_panel_ld.iloc[:,
                                           european_reference_panel_map_maf_filtered.index]
european_reference_panel_ld_maf_filtered = european_reference_panel_ld_maf_filtered.loc[
    european_reference_panel_map_maf_filtered.index]
#european_reference_panel_ld_maf_filtered = european_reference_panel_ld_maf_filtered.to_numpy()
#european_reference_panel_map_maf_filtered.reset_index(drop=True, inplace=True)
ld_filtered_reset_index = european_reference_panel_ld_maf_filtered.reset_index()

In [239]:
european_reference_panel_map_maf_filtered

,SNP_id,SNP_pos,POS19,Major,Minor,maf
0,rs969931,29602876,29494897,A,C,0.478086
1,rs2745406,29604062,29496083,T,C,0.468517
2,rs6939431,29604199,29496220,G,A,0.025263
3,rs1233427,29604643,29496664,G,A,0.421435
4,rs1233426,29606081,29498102,G,A,0.421435
...,...,...,...,...,...,...
8956,rs2744505,33261100,33153122,G,C,0.130813
8957,rs2855459,33262634,33154656,C,T,0.130718
8958,rs2855429,33266167,33158189,G,T,0.257895
8959,rs9469378,33267642,33159664,T,C,0.045167


In [240]:
temp = european_reference_panel_map_maf_filtered.merge(sample_summary, on='SNP_pos')
temp

,SNP_id_x,SNP_pos,POS19,Major,Minor,maf,SNP_id_y,Effect_allele,Non_effect_allele,T,T_CDF,T-P,T_2CDF,T_2CDF2,T_PPF,T-Z
0,rs969931,29602876,29494897,A,C,0.478086,rs969931,G,C,-2.8550,-2.957368,0.051955,-5.914736,-6.607883,-3.000048,-3.000048
1,rs909968,29607484,29499505,G,A,0.411196,rs909968,T,A,-2.4490,-2.702929,0.067009,-5.405858,-6.099005,-2.841499,-2.841499
2,rs3094575,29623781,29515802,C,T,0.256459,rs3094575,T,A,1.2560,-1.783926,0.167977,-3.567852,-4.260999,-2.194264,2.194264
3,rs3094576,29624221,29516242,C,A,0.123158,rs3094576,A,T,3.5990,-3.362793,0.034638,-6.725587,-7.418734,-3.238924,3.238924
4,rs376681,29625072,29517093,T,C,0.434641,rs376681,G,C,-2.6630,-2.840312,0.058407,-5.680624,-6.373771,-2.928007,-2.928007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
964,rs9277932,33249231,33141253,G,T,0.448995,rs9277932,T,A,-1.2310,-1.761884,0.171721,-3.523768,-4.216916,-2.176899,-2.176899
965,rs2855430,33249258,33141280,C,T,0.130813,rs2855430,A,T,1.2140,-1.746843,0.174323,-3.493686,-4.186833,-2.164987,2.164987
966,rs3762013,33250517,33142539,A,G,0.449091,rs3762013,G,C,-1.0700,-1.617887,0.198317,-3.235774,-3.928921,-2.060720,-2.060720
967,rs2855425,33252351,33144373,T,C,0.257703,rs2855425,G,C,-0.9776,-1.533940,0.215684,-3.067879,-3.761027,-1.990647,-1.990647


In [241]:
temp_pos = temp.query('(Major == Effect_allele & Minor == Non_effect_allele) | (Major == Non_effect_allele & Minor == Effect_allele)')['SNP_pos']
#temp_pos = temp_pos.sort_index(inplace=True)
temp_pos

50     29788636
53     29790845
739    32538340
741    32539270
Name: SNP_pos, dtype: int64

In [242]:
reset_for_typed_idx = european_reference_panel_map_maf_filtered.reset_index()
typed_idx = reset_for_typed_idx.loc[reset_for_typed_idx['SNP_pos'].isin(temp_pos)].index
typed_idx


Int64Index([206, 210, 6355, 6359], dtype='int64')

In [243]:
if len(typed_idx) >= 1:
    ld_tt = european_reference_panel_ld_maf_filtered.iloc[:,typed_idx]
    ld_tt = ld_tt.iloc[typed_idx, :]
    #ld_tt = ld_tt.loc[:, typed_idx]
    ld_tt = ld_tt + np.identity(len(ld_tt.columns)) * 0.15
    ld_it = european_reference_panel_ld_maf_filtered.drop(typed_idx, axis=0)
    ld_it = ld_it.iloc[:,typed_idx]
    map_t = european_reference_panel_map_maf_filtered.iloc[typed_idx]
    map_i = european_reference_panel_map_maf_filtered.drop(typed_idx, axis=0)

In [244]:
(ld_tt.shape, ld_it.shape, map_t.shape, map_i.shape)

((4, 4), (8440, 4), (4, 6), (8440, 6))

In [246]:
typed_in_both_idx = sample_summary.loc[sample_summary['SNP_pos'].isin(temp_pos)].index
typed_in_both_idx

Int64Index([185, 189, 1047, 1051], dtype='int64')

In [247]:
if len(typed_in_both_idx) >= 1:
    sample_summary = sample_summary.iloc[typed_in_both_idx]

Zt = np.full(len(sample_summary['T-Z']), None)
Zt

array([None, None, None, None], dtype=object)

In [248]:
sample_summary.iloc[2]

SNP_id               rs4428528
SNP_pos               32538340
Effect_allele                G
Non_effect_allele            C
T                       -5.122
T_CDF                -4.015507
T-P                   0.018034
T_2CDF               -8.031014
T_2CDF2              -8.724161
T_PPF                -3.594337
T-Z                  -3.594337
Name: 1047, dtype: object

In [249]:
for i in range(len(Zt)):
    #gwas_merged.iloc[i]['POS19']
    if sample_summary.iloc[i]['Effect_allele'] != map_t.loc[map_t[map_t['SNP_pos'] == sample_summary.iloc[i]['SNP_pos']].index].iloc[0]['Minor']:
        Zt[map_t['SNP_pos'] == sample_summary.iloc[i]['SNP_pos']] = -sample_summary.iloc[i]['T-Z']
    else:
        Zt[map_t['SNP_pos'] == sample_summary.iloc[i]['SNP_pos']] = sample_summary.iloc[i]['T-Z']

0
1
2
3


In [250]:
map_t.loc[map_t[map_t['SNP_pos'] == sample_summary.iloc[2]['SNP_pos']].index]

,SNP_id,SNP_pos,POS19,Major,Minor,maf
6738,rs4428528,32538340,32430362,C,G,0.264689


In [251]:
map_t

,SNP_id,SNP_pos,POS19,Major,Minor,maf
206,rs7741807,29788636,29680657,G,C,0.029187
210,rs1627567,29790845,29682866,A,T,0.277512
6738,rs4428528,32538340,32430362,C,G,0.264689
6742,rs9268878,32539270,32431292,T,A,0.264880


In [265]:
ld_tt

,V207,V211,V6739,V6743
206,1.150000,0.279768,-0.030579,-0.030647
210,0.279768,1.150000,-0.093783,-0.094030
6738,-0.030579,-0.093783,1.150000,0.999509
6742,-0.030647,-0.094030,0.999509,1.150000


In [266]:
la.det(ld_tt)

0.3995940718604002

In [267]:
np.linalg.cond(ld_tt)

14.442234931474117

In [268]:
ld_tt_inv = la.solve(ld_tt, np.identity(len(ld_tt.columns)))
ld_tt_inv

array([[ 9.24314905e-01, -2.24314858e-01,  3.33910110e-03,
         3.38920951e-03],
       [-2.24314858e-01,  9.30251446e-01,  3.67327576e-02,
         3.81587601e-02],
       [ 3.33910110e-03,  3.67327576e-02,  3.55668082e+00,
        -3.08815356e+00],
       [ 3.38920951e-03,  3.81587601e-02, -3.08815356e+00,
         3.55680684e+00]])

In [270]:
weight = np.dot(ld_it, ld_tt_inv)
weight

array([[ 0.05644689,  0.06359244,  0.04472987,  0.04773058],
       [ 0.05897275,  0.06191335,  0.04743777,  0.0504861 ],
       [-0.05303876,  0.1523436 , -0.02161271, -0.02175948],
       ...,
       [-0.03240565,  0.01510092,  0.0057568 ,  0.00405221],
       [ 0.02780723, -0.00990832, -0.024531  , -0.02507976],
       [ 0.00476624, -0.00207751,  0.00473479,  0.0066765 ]])

In [258]:
Zi = np.dot(weight, Zt)
Zi

array([-0.08420807960187399, -0.10554187711520255, 0.5224063237682237,
       ..., -0.03194976148011233, 0.18423707056876817,
       -0.04114112342529624], dtype=object)

In [259]:
var = np.full(len(weight), None)
var

array([None, None, None, ..., None, None, None], dtype=object)

In [286]:
for i in range(len(weight)):
    var[i] = np.sum(np.multiply(np.outer(weight[i], weight[i]), ld_tt.to_numpy()))
var

array([0.018088072487241404, 0.019264991363871754, 0.02852572125142223,
       ..., 0.0012913488126812915, 0.003485343016921119,
       0.00016690587995199413], dtype=object)

In [310]:
impute_p = lambda x: 2*st.norm.cdf(-abs(x))
imputed_p = impute_p(Zi.astype('float64'))
imputed_p

array([0.93289099, 0.91594584, 0.60138745, ..., 0.97451211, 0.85382748,
       0.96718339])

dtype('O')

In [311]:
results = {'Marker_id':map_i['SNP_id'], 'Marker_pos':map_i['SNP_pos'], 'Effect_allele':map_i['Minor'], 'Non_effect_allele':map_i['Major'], 'Imputed_Z':Zi, 'r2pred':var,'imputed_P': 2*st.norm.cdf(-abs(Zi.astype('float64')))}

In [298]:
results

[0        rs969931
 1       rs2745406
 2       rs6939431
 3       rs1233427
 4       rs1233426
           ...    
 8956    rs2744505
 8957    rs2855459
 8958    rs2855429
 8959    rs9469378
 8960    rs9277935
 Name: SNP_id, Length: 8440, dtype: object,
 0       29602876
 1       29604062
 2       29604199
 3       29604643
 4       29606081
           ...   
 8956    33261100
 8957    33262634
 8958    33266167
 8959    33267642
 8960    33268403
 Name: SNP_pos, Length: 8440, dtype: int64,
 0       C
 1       C
 2       A
 3       A
 4       A
        ..
 8956    C
 8957    T
 8958    T
 8959    C
 8960    T
 Name: Minor, Length: 8440, dtype: object,
 0       A
 1       T
 2       G
 3       G
 4       G
        ..
 8956    G
 8957    C
 8958    G
 8959    T
 8960    G
 Name: Major, Length: 8440, dtype: object,
 array([-0.08420807960187399, -0.10554187711520255, 0.5224063237682237,
        ..., -0.03194976148011233, 0.18423707056876817,
        -0.04114112342529624], dtype=object),
 ar

In [289]:
cols = ["Marker_id", "Marker_pos", "Effect_allele", "Non_effect_allele", "Imputed_Z",  "r2pred", "imputed_P"]

In [312]:
df = pd.DataFrame(results)
df

,Marker_id,Marker_pos,Effect_allele,Non_effect_allele,Imputed_Z,r2pred,imputed_P
0,rs969931,29602876,C,A,-0.084208,0.018088,0.932891
1,rs2745406,29604062,C,T,-0.105542,0.019265,0.915946
2,rs6939431,29604199,A,G,0.522406,0.028526,0.601387
3,rs1233427,29604643,A,G,-0.281259,0.017927,0.778512
4,rs1233426,29606081,A,G,-0.281259,0.017927,0.778512
...,...,...,...,...,...,...,...
8956,rs2744505,33261100,C,G,-0.092582,0.001178,0.926235
8957,rs2855459,33262634,T,C,-0.094785,0.001235,0.924485
8958,rs2855429,33266167,T,G,-0.03195,0.001291,0.974512
8959,rs9469378,33267642,C,T,0.184237,0.003485,0.853827


In [314]:
df.to_csv('./results/sample_results.tsv', '\t')